In [1]:
import pandas as pd
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls

drive  sample_data


In [4]:
df_tweet_train = pd.read_csv("/content/drive/MyDrive/Program Platinum Challange/train_preprocess.tsv.txt",header=None,names=['tweet','sentiment'],delimiter='\t')
df_tweet_train

,tweet,sentiment
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative
...,...,...
10995,tidak kecewa,positive
10996,enak rasa masakan nya apalagi kepiting yang me...,positive
10997,hormati partai-partai yang telah berkoalisi,neutral
10998,"pagi pagi di tol pasteur sudah macet parah , b...",negative


In [5]:
!pip3 install nltk
!pip3 install Sastrawi
!pip3 install nlp_id

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.5/320.5 kB 34.7 MB/s eta 0:00:00
  Created wheel for nlp_id: filename=nlp_id-0.1.14.0-py3-none-any.whl size=8074110 sha256=dda9c77cf993b4681a5aafe403318e7b6bf5d4bc8b2e929c35bbb35ba15fff46
  Stored in directory: /root/.cache/pip/wheels/16/5d/1e/051d2642dfffeb89716569b4dad1552b6e8dd980572c5e3496
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=a94ed1ed479638b8ebab549e9043540ea9f408ed75131dd79eda08644183ec51
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built nlp_id wget
  Attempting uninstall: pytest
    Found existing installation: pytest 7.2.2
    Uninstalling pytest-7.2.2:
      Successfully uninsta

In [6]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nlp_id.lemmatizer import Lemmatizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

alay_dict=pd.read_csv("/content/drive/MyDrive/Program Platinum Challange/DATA/new_kamusalay.csv",encoding='latin1',header=None)
alay_dict=alay_dict.rename(columns={0:'Original',1:'Baku'})

kasar_dict=pd.read_csv("/content/drive/MyDrive/Program Platinum Challange/DATA/abusive.csv",encoding='latin1')
kasar_dict['Kata_Sensor']="a1b2c3d4e5f6" #Inisiasi kata ganti untuk kata-kata yang kasar dengan kata "disensor"

print(kasar_dict)

alay_dict_map = dict(zip(alay_dict['Original'], alay_dict['Baku']))
kasar_dict_map = dict(zip(kasar_dict['ABUSIVE'],kasar_dict['Kata_Sensor']))

def normalize_alay(text):
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])


def sensor_kata_kasar(text):
    return ' '.join([kasar_dict_map[word] if word in kasar_dict_map else word for word in text.split(' ')])


def preprocess(TextYangInginDiPreProcess):
    #Tahap Pertama Adalah Membuat semua huruf menjadi huruf kecil atau lower
    text = TextYangInginDiPreProcess.lower()

    #Tahap Kedua adalah menghilangkan non alpha numeric character pada text
    text = re.sub('[^0-9a-zA-Z]+',' ',text)

    #Tahap Ketiga adalah menghilangkan char tidak penting
    text=re.sub('\n',' ',text) #Menghilangkan new line pada data
    text=re.sub('rt',' ',text) #Menghilangkan kata-kata retweet
    text=re.sub('user',' ',text) #Menghilangkan kata-kata user
    text=re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text) #Menghilangkan  URL
    text=re.sub(' +',' ',text) #Menghilangkan ekstra spasi

    #Tahap keempat adalah membuat map terhadap kata-kata "alay" dan mengubah nya menjadi kata yang baku
    text=normalize_alay(text)

    #Tahap kelima adalah mensensor kata kasar dengan kata "disensor"
    text=sensor_kata_kasar(text)
    return text

def menghilangkan_kata_umum_dengan_stopword(text):
    # Mengambil daftar stopwords bahasa Indonesia dari NLTK
    stop_words = set(stopwords.words('indonesian'))

    # Memisahkan teks menjadi kata-kata
    words = text.split()

    # Menghapus stopwords dari teks
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # Menggabungkan kata-kata yang tersisa menjadi teks tanpa stopwords
    filtered_text = ' '.join(filtered_words)

    return filtered_text

def mengurangi_variasi_kata_dengan_steeming(teks):
    Stemming = StemmerFactory()
    Fact = Stemming.create_stemmer()
    teks = Fact.stem(teks)
    return teks

def preprocess_semua_kata(teks):
    teks = preprocess(teks)
    teks = menghilangkan_kata_umum_dengan_stopword(teks)
    teks = mengurangi_variasi_kata_dengan_steeming(teks)
    return teks

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


         ABUSIVE   Kata_Sensor
0           alay  a1b2c3d4e5f6
1          ampas  a1b2c3d4e5f6
2           buta  a1b2c3d4e5f6
3        keparat  a1b2c3d4e5f6
4         anjing  a1b2c3d4e5f6
..           ...           ...
120        rezim  a1b2c3d4e5f6
121        sange  a1b2c3d4e5f6
122       serbet  a1b2c3d4e5f6
123        sipit  a1b2c3d4e5f6
124  transgender  a1b2c3d4e5f6

[125 rows x 2 columns]


In [7]:
kata_kata = "Dasar Anak Anjing"
print(preprocess_semua_kata(kata_kata))


dasar anak a1b2c3d4e5f6


In [8]:
pos_sent = df_tweet_train[df_tweet_train['sentiment']=='positive']
pos_sent

,tweet,sentiment
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
5,"makanan beragam , harga makanan di food stall ...",positive
7,"tempat unik , bagus buat foto , makanan enak ,...",positive
...,...,...
10992,kalau memang jala nya sudah begitu terima saja...,positive
10994,tidak bosan,positive
10995,tidak kecewa,positive
10996,enak rasa masakan nya apalagi kepiting yang me...,positive


In [9]:
neu_sent=df_tweet_train[df_tweet_train['sentiment']=='neutral']
neu_sent

,tweet,sentiment
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
16,setiap bumn dibentuk dengan uu bukan dibentuk ...,neutral
28,pdip sebut ridwan kamil menang karena berbaju ...,neutral
32,08:30 : kedatangan presiden ri dan rombongan d...,neutral
37,risma menjatuhkan pilihan nya pada gus ipul - ...,neutral
...,...,...
10925,pkb benarkan charly setia band daftar jadi caw...,neutral
10967,pernah percaya kalau body lotion citra bikin b...,neutral
10969,"bu susi marah , menteri luhut perbolehkan kapa...",neutral
10993,f - demokrat dorong upaya kemandirian energi n...,neutral


In [10]:
neg_sent = df_tweet_train[df_tweet_train['sentiment']=='negative']
neg_sent

,tweet,sentiment
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative
6,pakai kartu kredit bca tidak untung malah rugi...,negative
10,simcard indosat inaktiv gara-gara lupa isi pul...,negative
11,sifat iri sering muncul pada orang orang yang ...,negative
13,"pengalaman bersama indosat hari ini , semoga t...",negative
...,...,...
10982,tolong dibaca pesan saya ! di dalam kotak hand...,negative
10985,"sumpah gue kecewa banget dengan sms , cuma tea...",negative
10989,tadinya pengin mengantar teman dari kalau ke r...,negative
10991,dapat rom awal abad,negative


In [11]:
pos_sent = pos_sent[:neu_sent.shape[0]]
neg_sent = neg_sent[:neu_sent.shape[0]]
pos_sent

,tweet,sentiment
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
5,"makanan beragam , harga makanan di food stall ...",positive
7,"tempat unik , bagus buat foto , makanan enak ,...",positive
...,...,...
1923,tidak keberatan,positive
1924,"setiap kali saya makan kemari , saya selalu me...",positive
1925,steak yang paling saya sukai di bandung . disa...,positive
1926,"tempat nya homy , terang dengan dekorasi seper...",positive


In [12]:
neg_sent

,tweet,sentiment
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative
6,pakai kartu kredit bca tidak untung malah rugi...,negative
10,simcard indosat inaktiv gara-gara lupa isi pul...,negative
11,sifat iri sering muncul pada orang orang yang ...,negative
13,"pengalaman bersama indosat hari ini , semoga t...",negative
...,...,...
3756,memang babi anjinglah orang-orang sarawak bang...,negative
3761,saya iri fungsi sekolah telah berubah seperti ini,negative
3764,"halo j triliun saya mau tanya , kenapa butuh m...",negative
3773,"lu agama buddha sesat , terserah ! tidak penti...",negative


In [13]:
df_baru_gabungan = pd.concat([pos_sent,neu_sent,neg_sent])

In [14]:
df_baru_gabungan = df_baru_gabungan.reset_index(drop=True)

In [15]:
df_baru_gabungan

,tweet,sentiment
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,lokasi strategis di jalan sumatera bandung . t...,positive
2,betapa bahagia nya diri ini saat unboxing pake...,positive
3,"makanan beragam , harga makanan di food stall ...",positive
4,"tempat unik , bagus buat foto , makanan enak ,...",positive
...,...,...
3439,memang babi anjinglah orang-orang sarawak bang...,negative
3440,saya iri fungsi sekolah telah berubah seperti ini,negative
3441,"halo j triliun saya mau tanya , kenapa butuh m...",negative
3442,"lu agama buddha sesat , terserah ! tidak penti...",negative


In [16]:
#df_baru_gabungan['tweet'] = df_baru_gabungan['tweet'].apply(preprocess_semua_kata)

In [17]:
#df_baru_gabungan

In [18]:
#df_baru_gabungan.to_csv("/content/drive/MyDrive/Program Platinum Challange/"+"Data_Yang_Telah_Di_Preprocess.csv")

In [19]:
df_testing = pd.read_csv("/content/drive/MyDrive/Program Platinum Challange/DATA/data.csv",encoding='latin1')

In [20]:
df_testing

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13164,USER jangan asal ngomong ndasmu. congor lu yg ...,1,1,1,0,0,0,1,0,0,1,0,0
13165,USER Kasur mana enak kunyuk',0,1,0,0,0,0,0,0,0,0,0,0
13166,USER Hati hati bisu :( .g\n\nlagi bosan huft \...,0,0,0,0,0,0,0,0,0,0,0,0
13167,USER USER USER USER Bom yang real mudah terdet...,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
df_testing_baru = df_testing['Tweet'].apply(preprocess_semua_kata)

In [22]:
df_testing_baru.to_csv("/content/drive/MyDrive/Program Platinum Challange/"+"Data_Testing_Yang_Telah_Di_Preprocess.csv")